In [ ]:
from pycoingecko import CoinGeckoAPI
import pandas as pd
from datetime import datetime
import os
import base64
import requests
from io import BytesIO

# Initialize API
cg = CoinGeckoAPI()

# List of coins and their CoinGecko IDs
coins = {
    'BTC': 'bitcoin',
    'ETH': 'ethereum',
    'ADA': 'cardano',
    'SOL': 'solana',
    'DOT': 'polkadot',
    'AVAX': 'avalanche-2'
}

# Fetch volume data from market chart endpoint
def fetch_volume(coin_id):
    market_data = cg.get_coin_market_chart_by_id(id=coin_id, vs_currency='usd', days=365)
    volume_data = market_data['total_volumes']
    df = pd.DataFrame(volume_data, columns=['timestamp', 'volume'])
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms').dt.date
    df.drop(columns=['timestamp'], inplace=True)
    return df

# Collect data for all coins
volume_data = {}
for symbol, coin_id in coins.items():
    df = fetch_volume(coin_id)
    df['symbol'] = symbol
    volume_data[symbol] = df

# Combine all into one DataFrame
new_data_df = pd.concat(volume_data.values(), ignore_index=True)

# Airtable setup
AIRTABLE_API_KEY = os.getenv("AIRTABLE_API_KEY")
BASE_ID = "appnssPRD9yeYJJe5"
TABLE_NAME = "Database"
airtable_url = f"https://api.airtable.com/v0/{BASE_ID}/{TABLE_NAME}"

airtable_headers = {
    "Authorization": f"Bearer {AIRTABLE_API_KEY}",
    "Content-Type": "application/json"
}

# Fetch current records
response = requests.get(airtable_url, headers=airtable_headers)
response.raise_for_status()
data = response.json()

# Check for existing volume data file
existing_records = [
    rec for rec in data['records']
    if rec['fields'].get('Name') == "Volume Traded Master Data"
]
if existing_records and 'Database Attachment' in existing_records[0]['fields']:
    attachment_url = existing_records[0]['fields']['Database Attachment'][0]['url']
    existing_file = requests.get(attachment_url)
    existing_file.raise_for_status()
    existing_df = pd.read_excel(BytesIO(existing_file.content))
    combined_df = pd.concat([existing_df, new_data_df], ignore_index=True)
else:
    combined_df = new_data_df

# Save to Excel
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"crypto_volume_traded_with_history_{timestamp}.xlsx"
combined_df.to_excel(filename, index=False)

# GitHub Upload
GITHUB_REPO = "SagarFieldElevate/DatabaseManagement"
BRANCH = "main"
UPLOAD_PATH = "uploads"
GITHUB_TOKEN = os.getenv("GH_TOKEN")

with open(filename, "rb") as f:
    content = base64.b64encode(f.read()).decode()

upload_url = f"https://api.github.com/repos/{GITHUB_REPO}/contents/{UPLOAD_PATH}/{filename}"
upload_payload = {
    "message": f"Upload Volume Traded data with history {timestamp}",
    "content": content,
    "branch": BRANCH
}
headers = {
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json"
}
upload_resp = requests.put(upload_url, headers=headers, json=upload_payload)
if upload_resp.status_code not in [200, 201]:
    raise Exception(f"❌ GitHub upload failed: {upload_resp.status_code} - {upload_resp.text}")

raw_url = f"https://raw.githubusercontent.com/{GITHUB_REPO}/{BRANCH}/{UPLOAD_PATH}/{filename}"

# Update or create Airtable record
if existing_records:
    record_id = existing_records[0]['id']
    patch_url = f"{airtable_url}/{record_id}"
    patch_payload = {
        "fields": {
            "Database Attachment": [{
                "url": raw_url,
                "filename": filename
            }]
        }
    }
    airtable_resp = requests.patch(patch_url, headers=airtable_headers, json=patch_payload)
else:
    post_payload = {
        "records": [{
            "fields": {
                "Name": "Volume Traded Master Data",
                "Database Attachment": [{
                    "url": raw_url,
                    "filename": filename
                }]
            }
        }]
    }
    airtable_resp = requests.post(airtable_url, headers=airtable_headers, json=post_payload)

if airtable_resp.status_code != 200:
    raise Exception(f"❌ Airtable upload failed: {airtable_resp.status_code} - {airtable_resp.text}")

# Delete the file from GitHub after upload
delete_url = f"https://api.github.com/repos/{GITHUB_REPO}/contents/{UPLOAD_PATH}/{filename}"
file_sha = upload_resp.json()['content']['sha']  # Get the SHA of the uploaded file
delete_payload = {
    "message": f"Delete 365-day volatility and trading range file {timestamp}",
    "sha": file_sha
}

delete_resp = requests.delete(delete_url, headers=headers, json=delete_payload)
if delete_resp.status_code != 200:
    raise Exception(f"❌ GitHub file deletion failed: {delete_resp.status_code} - {delete_resp.text}")

# Delete local file
os.remove(filename)
print("🧽 Local file and GitHub file deleted.")



🧽 Local file deleted.
